#### Pre-requisites

 1. Python runtime should be installed in you system. Link for Python 3.11.0(https://www.python.org/downloads/release/python-3110/)
 2. Install python packages given in the next cell.


In [1]:
# %%capture
# !pip install openai
# !pip install python-dotenv
# !pip install urllib3

Importing the necessary packages along with the keys in our .env file that you created.

In [1]:
import json
import urllib3
import os
from dotenv import load_dotenv
from prompts import formatter_prompt,assistant_instructions
from openai import OpenAI
import time

load_dotenv()

GOOGLE_API = os.getenv('GOOGLE_API')
AIRTABLE_API = os.getenv('AIRTABLE_API')
AIRTABLE_URL = os.getenv('AIRTABLE_URL')

#### The class AssistantManager will store functions for creating assistants, creating threads for the assistants, adding message to the threads, running the assistants. Then we will wait for the the completions and submit the response back to the assistant again for generating a response.

In [3]:
class AssistantManager:
  def __init__(self,api_key:str,model:str="gpt-3.5-turbo-1106"):
    self.client = OpenAI(api_key=api_key)
    self.model = model
    self.assistant = None
    self.thread = None
    self.run = None
    self.file = None
    self.message = None

  def python_assistant(self,name, instructions, tools):
        self.assistant = self.client.beta.assistants.create(
            name = name,
            instructions = instructions,
            tools = tools,
            model = self.model
        )

  def create_assistant(self,name, instructions, tools):
        self.file = self.client.files.create(file=open("knowledge.docx","rb"),purpose='assistants')
        self.assistant = self.client.beta.assistants.create(
            name = name,
            instructions = instructions,
            tools = tools,
            model = self.model,
            file_ids = [self.file.id]
        )

  def create_thread(self):
      self.thread = self.client.beta.threads.create()

  def add_message_to_thread(self,role,content):
    self.message = self.client.beta.threads.messages.create(
      thread_id = self.thread.id,
      role = role,
      content = content
    )

  def run_assistant(self,instructions):
    self.run = self.client.beta.threads.runs.create(
      thread_id = self.thread.id,
      assistant_id=self.assistant.id,
      instructions = instructions
    )

  def process_messages(self):
    messages = self.client.beta.threads.messages.list(thread_id = self.thread.id)

    msg = messages.data[0]
    role = msg.role
    content = msg.content[0].text.value
    print(f"{role.capitalize}:{content}")

  def wait_for_completion(self):
    while True:
      time.sleep(5)
      run_status = self.client.beta.threads.runs.retrieve(
        thread_id = self.thread.id,
        run_id = self.run.id
      )
      
      print(run_status.status)
      if run_status.status == 'completed':
        self.process_messages()
        break
      elif run_status.status == 'requires_action':
        print("Function Calling ...")
        self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
      elif run_status.status=="failed":
        print(run_status.last_error)
      else:
        print("Waiting for the Assistant to process..") 
  
  def call_required_functions(self, required_actions):
    tool_output = []

    for action in required_actions["tool_calls"]:
      func_name = action['function']['name']
      arguments = json.loads(action['function']['arguments'])
    
      func_name = eval(func_name)
      output = func_name(**arguments)
      tool_output.append(
        {
          "tool_call_id":action['id'],
          "output": str(output)
        }
      )
    
    print("Submitting outputs back to the Assistants...")
    self.client.beta.threads.runs.submit_tool_outputs(
      thread_id = self.thread.id,
      run_id = self.run.id,
      tool_outputs=tool_output
    )

#### Create lead function is used by the OpenAI Assistant for creating leads in you company CRM.(Here in our Airtable base)

In [4]:
def create_lead(name, phone, address):
    # Airtable API URL
    url = AIRTABLE_URL

    # Headers with Authorization and Content-Type
    headers = {
        "Authorization": AIRTABLE_API,  # Takes Airtable API from environment
        "Content-Type": "application/json"
    }

    # Data to be sent in the request
    data = {
        "records": [{
            "fields": {
                "Name": name,
                "Phone": phone,
                "Address": address,
                "Status": "Arrived"
            }
        }]
    }

    # Create a PoolManager for HTTP requests
    http = urllib3.PoolManager()

    # Encode data to bytes
    encoded_data = json.dumps(data).encode('utf-8')

    # Make a POST request to Airtable API
    response = http.request(
        'POST',
        url,
        body=encoded_data,
        headers=headers
    )

    # Check the response status
    if response.status == 200:
        print("Lead created successfully.")
        return response
    else:
        print(f"Failed to create lead: {response.data}")


#### Here the Google API is used for getting the longitude and latitude of the Location given by the user to generate solar potential

In [5]:
# Example usage:
# create_lead("John Doe", "1234567890", "123 Main St")
def get_coordinates(address):
  geocoding_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_API}"
  # response = requests.get(geocoding_url)

  timeout = urllib3.Timeout(connect=5.0, read=15.0)
  http = urllib3.PoolManager(timeout = timeout)

  # Make a POST request to Airtable API
  response = http.request(
        'GET',
        geocoding_url
    )

  if response.status == 200:
    response = response.data
    response = response.decode('utf-8')
    response = json.loads(response)
    location = response['results'][0]['geometry']['location']
    print(f"Coordinates for {address}: {location}")
    return location['lat'], location['lng']
  else:
    print(f"Error getting coordinates: {response.text}")
  # return '100.14.11.E.12','100.14.11.e.15'

#### After passing the longitude and latitude, the solar data is generated for the given location, the extract financial analyes will get the financial analyses from the solar data.

In [6]:
def get_solar_data(lat, lng):
  # The solar data for a given latitude and longitude is retrieved using this URL
  solar_api_url = f"https://solar.googleapis.com/v1/buildingInsights:findClosest?location.latitude={lat}&location.longitude={lng}&requiredQuality=HIGH&key={GOOGLE_API}"

  timeout = urllib3.Timeout(connect=5.0, read=15.0)
  http = urllib3.PoolManager(timeout=timeout)

  response = http.request(
        'GET',
        solar_api_url
    )
  
  if response.status == 200:
    response = response.data
    response = response.decode('utf-8')
    print("Solar data retrieved successfully.")
    response = json.loads(response)
    return response
  else:
    print(f"Error getting solar data: {response.text}")

def extract_financial_analyses(solar_data):
  try:
    # Getting the financial analyses from the solar data.
    return solar_data.get('solarPotential', {}).get('financialAnalyses', [])
  except KeyError as e:
    print(f"Data extraction error: {e}")

def get_financial_data_for_address(address):
  lat, lng = get_coordinates(address)
  if not lat or not lng:
    return {"error": "Could not get coordinates for the address provided."}
  return extract_financial_analyses(get_solar_data(lat, lng))

#### This function provides the closest match for the monthly bill that the user provided.

In [7]:
def find_closest_financial_analysis(user_bill, financial_analyses):
  closest_match = None
  smallest_difference = float('inf')
  for analysis in financial_analyses:
      # Accessing the nested dictionary
      monthly_bill_data = analysis.get('monthlyBill', {})

      # Extracting units
      bill_amount = int(monthly_bill_data.get('units', 0))
      difference = abs(bill_amount - user_bill)

      if difference < smallest_difference:
        smallest_difference = difference
        closest_match = analysis
  return closest_match

#### The simplify financial data returns the financial data in a fixed format

In [8]:
def simplify_financial_data(data):
  try:

    data_str = json.dumps(data, indent=2)

    # Getting formatter prompt from "prompts.py" file
    system_prompt = formatter_prompt
    api_key = os.getenv("Open_AI_key")

    client = OpenAI(api_key= api_key)
    # Replace 'client' with your actual OpenAI client initialization.
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content":
                system_prompt  # Getting prompt from "prompts.py" file
            },
            {
                "role":
                "user",
                "content":
                f"Here is some data, parse and format it exactly as shown in the example: {data_str}"
            }
        ],
        temperature=0)

    simplified_data = json.loads(completion.choices[0].message.content)
    return simplified_data

  except Exception as e:
    print("Error simplifying data:", e)
    return None

def solar_panel_calculations(address, monthly_bill):

  financial_analyses = get_financial_data_for_address(address)
  if "error" in financial_analyses:
    print(financial_analyses["error"])
    return financial_analyses
  monthly_bill = [i for i in str(monthly_bill) if i.isdigit() or i == '.']

  monthly_bill = float(''.join(monthly_bill))
  closest_financial_analysis = find_closest_financial_analysis(
      int(monthly_bill), financial_analyses)
  if closest_financial_analysis:
    return simplify_financial_data(closest_financial_analysis)
  else:
    print("No suitable financial analysis found.")
    return {
        "error": "No suitable financial analysis found for the given bill."
    }

#### Defining the functions is not enough, explaining the functions to the Assistants is key, which is done in terms of tools, that will store the function descriptions, function name, parameters and their descriptions.

In [9]:
tools = [{"type":"retrieval"},
        {
            "type": "function",
            "function": {
                "name": "solar_panel_calculations",
                "description": "Calculate solar potential based on a given address and monthly electricity bill in USD. Returns financial analysis.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "address": {
                            "type": "string",
                            "description": "Address for calculating solar potential.",
                        },
                        "monthly_bill":{
                            "type":"integer",
                            "description": "Monthly Bills to be generated.For example: 700 for 700 USD monthly bill"
                        }
                    },
                    "required": ["address","monthly_bill"],
                },
            }
        },
        {
        "type":"function", 
        "function":{
                "name": "create_lead",
                "description": "Create a lead using the name , phone number and address.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name":{
                              "type":"string",
                              "description":"Name of the lead."
                          },
                        "phone":{
                              "type":"string",
                              "description":"Phone number of the lead."
                          },
                        "address":{
                              "type":"string",
                              "description":"Address of the lead."
                        }
                    },
                    "required": ["name","phone","address"],
                },
            }
        }
    ]

#### The instructions below will provide the Assistants with context and how to respond to the query

In [10]:
instruction1 = '''
Greet the User if they say 'Hi' or 'Hey' or something similar.
If the user asks what the assistant is capable of, the assistant should be short and precise with their response and talk about the assistant's functionalities.

If the user wants to calculate solar potential, and provides the address and the monthly bill, the assistant should take this information to calulate solar potential. 
If the user does not provide address or monthly bill, the assistant should request the user for it. The assistant should not call the calculate solar potential function unless it has all the properties it needs.
If the address and monthly bill is provided, go ahead and call the function solar_panel_calculations and return the detailed calculations to the user along with a question in the end as "Please provide your name, phone number and address for contacting you".

If the user provides the name, phone number and address for creating lead, the assistant should take this information to create lead.
If the user does not give name or phone number or address, the assistant should request the user to provide it. The assistant should not call the create lead function unless it has all the properties it needs.
Once it has all the information, it can go ahead and call the function create_lead. If the lead is created successfully, the assistant should say "Thank you for providing your information. Someone from the team will get in touch with you to discuss further details."

Assistant should strictly follow this:
Do not ask for name, address, phone number before calculating solar potential
The assistant is not capable of making up random values for any missing information. The assistant should always have all the information it needs before calling the function. The assistant should remember the context of a question.
Extract the monthly bill without currency, for example: if user gives monthly bill as "300 USD" the assistant should extract "300" or "$300" should be extracted as "300"
Assistant should also follow this as side note:
If no suitable financial analysis is found, then assistant should say "I apologize, but it seems that no suitable financial analysis was found for the given monthly bill.This could maybe happen because of missing solar data on your location or the monthly bill is not suitable."
'''

instruction2 = '''
The assistant is a solar panel financial advisor, its job is to extract address and monthly bills for calling functions to calculate solar potential. 
The assistant can also ask for name, phone number and address for lead generation after the solar potential is generated.
'''

#### Creating the assistant for OpenAI

In [11]:
api_key = os.getenv("Open_AI_key")
manager = AssistantManager(api_key)

manager.create_assistant(
    name = "LeadGen",
    instructions=assistant_instructions,
    tools = tools
)

#### Creating a thread for the assistant adding message to the thread and waiting for the response.

In [12]:
time.sleep(5)
manager.create_thread()

manager.add_message_to_thread(role="user",content="Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 500 USD")

manager.run_assistant(instruction1)
manager.wait_for_completion()

requires_action
Function Calling ...
Coordinates for 9813 Sherborne Ave, Bakersfield, CA 93311, USA: {'lat': 35.3098072, 'lng': -119.1115797}
Solar data retrieved successfully.
Submitting outputs back to the Assistants...
in_progress
Waiting for the Assistant to process..
completed
<built-in method capitalize of str object at 0x0000022A5B3FC030>:Based on the provided details, here are the calculations for the solar potential:

- Monthly Bill: $500
- Federal Incentive: $9,872
- State Incentive: None Found
- Utility Incentive: None Found
- Total Cost Without Solar: $150,074
- Solar Coverage Percentage: 98.27%
- Leasing Option:
  - Annual Cost: $2,230
  - First Year Savings: $3,452
  - Twenty-Year Savings: $93,069
  - Present Value Twenty-Year: $61,976
- Cash Purchase Option:
  - Out Of Pocket Cost: $37,969.50
  - Payback Years: 4.75
  - First Year Savings: $5,682
  - Twenty-Year Savings: $137,680
  - Present Value Twenty-Year: $65,405
- Financed Purchase Option:
  - Annual Loan Payment: 

In [13]:
time.sleep(5)
# manager.add_message_to_thread(role="user",content="Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 300 USD")
manager.add_message_to_thread(role="user",content="Jacob Mane, 43th Avenue, California, 33452213998")

manager.run_assistant(instruction1)
manager.wait_for_completion()

requires_action
Function Calling ...
Lead created successfully.
Submitting outputs back to the Assistants...
failed
LastError(code='rate_limit_exceeded', message='Rate limit reached for gpt-3.5-turbo-1106 in organization org-FzJQSaGLr5BP3G7DfJTSJVeA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.')
failed
LastError(code='rate_limit_exceeded', message='Rate limit reached for gpt-3.5-turbo-1106 in organization org-FzJQSaGLr5BP3G7DfJTSJVeA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.')
failed
LastError(code='rate_limit_exceeded', message='R

In [ ]:
time.sleep(5)
manager.add_message_to_thread(role="user",content="Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 500 USD and create lead for Jacob Mane, 43th Avenue, California, 33452213998")

manager.run_assistant(instruction1)
manager.wait_for_completion()

in_progress
Waiting for the Assistant to process..
requires_action
Function Calling ...
Coordinates for 9813 Sherborne Ave, Bakersfield, CA 93311, USA: {'lat': 35.3098072, 'lng': -119.1115797}
Solar data retrieved successfully.
Lead created successfully.
Submitting outputs back to the Assistants...
in_progress
Waiting for the Assistant to process..
in_progress
Waiting for the Assistant to process..
completed
<built-in method capitalize of str object at 0x0000019FAACE10B0>:The solar potential calculations for the address "9813 Sherborne Ave, Bakersfield, CA 93311, USA" with a monthly bill of $500 are as follows:

- Monthly Bill: $500
- Federal Incentive: $9,872
- State Incentive: None Found
- Utility Incentive: None Found
- Total Cost Without Solar: $150,074
- Solar Coverage Percentage: 98.27%
- Leasing Option:
  - Annual Cost: $2,230
  - First Year Savings: $3,452
  - Twenty Year Savings: $93,069
  - Present Value over Twenty Years: $61,976
- Cash Purchase Option:
  - Out of Pocket Cos

## Code Interpreter using OpenAI Assistants

In [ ]:
code_instruction = '''you have the user_bill as integer and financial analysis which is a list of json objects that contains the monthlyBill and inside monthlyBill the units is stored, 
you need to write a code in python which calculates the absolute difference between the units and the user_bill that is stored in smallest_difference if it is smaller than the previous smallest difference 
and for the smallest difference the array object is stored in a variable called closest_match and the variable is returned.'''

code_run_instruction = '''Run the function with user_bill = <user_bill> and financial_analysis = <financial_analysis>. Display the code for the function and the entire array object stored in closest match'''

In [ ]:
code_obj = AssistantManager(os.getenv("Open_AI_key"))

code_obj.python_assistant(
    name = "Python Developer",
    instructions="You are a Senior Python developer and you job is to create ,run and display python codes and return the outputs",
    tools = [{'type':'code_interpreter'}]
)

### As we have seen above that we try to find the financial analysis that is closer to the monthly bill of the user. Above we did this using manually coded function, but this can be done using the Code interpreter tool of OpenAI Assistants. Here we generated the same code using Assistants, rather than manually coding it, displaying the power of Code Interpreter with Assistants. 

In [ ]:
time.sleep(5)
closest_match = ''
result =  [
    {"monthlyBill": {"units": 120}},
    {"monthlyBill": {"units": 160}},
    {"monthlyBill": {"units": 140}},
    # Add more entries as needed
]
user_bill = 100

code_obj.create_thread()

code_obj.add_message_to_thread(role="user",content=code_instruction)
code_obj.run_assistant(code_run_instruction.replace('<user_bill>',str(user_bill)).replace('<financial_analysis>',str(result)))

while True:
    time.sleep(5)
    run_status = code_obj.client.beta.threads.runs.retrieve(
        thread_id = code_obj.thread.id,
        run_id = code_obj.run.id
    )
    
    print(run_status.status)
    if run_status.status == 'completed':
        messages = code_obj.client.beta.threads.messages.list(thread_id = code_obj.thread.id)
        # for msg in messages.data:
        #   role = msg.role
        msg = messages.data[0]
        role = msg.role
        content = msg.content[0].text.value
        print(content)
        index = content.find('{')
        print(closest_match)
        break
    elif run_status.status=="failed":
        print(run_status.last_error)
    else:
        print("Waiting for the Assistant to process..")

in_progress
Waiting for the Assistant to process..
in_progress
Waiting for the Assistant to process..
completed
The code for the function is as follows:

```python
def find_closest_match(user_bill, financial_analysis):
    smallest_difference = float('inf')
    closest_match = None
    for entry in financial_analysis:
        units = entry.get('monthlyBill', {}).get('units', 0)
        difference = abs(units - user_bill)
        if difference < smallest_difference:
            smallest_difference = difference
            closest_match = entry
    return closest_match
```

When the function is called with `user_bill = 100` and `financial_analysis = [{'monthlyBill': {'units': 120}}, {'monthlyBill': {'units': 160}}, {'monthlyBill': {'units': 140}}]`, the closest match is found to be `{'monthlyBill': {'units': 120}}`.

